In [ ]:
import pyodbc
import pandas as pd

# Path to the Access database file
mdb_file = r"C:\Users\gia.luongdo\Desktop\ERP-Importer\db_Artikel_Export2.mdb"
conn_str = (
    query = """
SELECT 
    m.ArtBasis AS aid,
    sku.ArtikelCode AS variant_aid,
    'classification_system' AS classification_system,
    'attribute[0]' AS [attribute[0]],
    '' AS [attribute_value[0]],
    '' AS [is_mandatory[0]],
    '' AS production_type
FROM t_Art_Mega_SKU sku
INNER JOIN t_Art_MegaBase m ON sku.ArtNr = m.ArtNr
WHERE m.Marke IN ('Corporate', 'EXCD', 'XO')
"""
df = pd.read_sql(query, conn)
    f"DBQ={mdb_file};"
)

# Connect to Access database
conn = pyodbc.connect(conn_str)

# Query data from table
query = "SELECT * FROM t_Art_Mega_SKU"
df = pd.read_sql(query, conn)

# Export to CSV file
df.to_csv(r"C:\Users\gia.luongdo\Desktop\ERP-Importer\exported_data.csv", index=False, encoding='utf-8', sep=',')

conn.close()


In [ ]:
import pandas as pd

# --- Đọc dữ liệu ---
file_to_compare = r"C:\Users\gia.luongdo\Desktop\AID_SQL_.csv"
column1_name = 'aid'
column2_name = 'AID'
csv_delimiter = ','

try:
    df = pd.read_csv(file_to_compare, delimiter=csv_delimiter)

    # Kiểm tra sự tồn tại của các cột cần thiết
    if column1_name not in df.columns or column2_name not in df.columns:
        print(f"Lỗi: Thiếu một trong hai cột '{column1_name}' hoặc '{column2_name}'. Các cột có sẵn: {df.columns.tolist()}")
        raise SystemExit

    aid_list = df[column1_name].dropna().astype(str).str.strip().tolist()
    AID_list = df[column2_name].dropna().astype(str).str.strip().tolist()

    khong_trung_khop = []
    for aid in aid_list:
        if aid not in AID_list:
            khong_trung_khop.append(aid)

    print(f"Tổng số giá trị kiểm tra trong '{column1_name}': {len(aid_list)}")
    print(f"Số lượng giá trị trong '{column2_name}': {len(AID_list)}")

    if khong_trung_khop:
        print(f"\nCác giá trị trong '{column1_name}' KHÔNG trùng khớp với giá trị nào trong '{column2_name}':")
        for v in khong_trung_khop:
            print(v)
        print(f"Tổng cộng: {len(khong_trung_khop)}")
    else:
        print(f"Tất cả giá trị trong '{column1_name}' đều có trong '{column2_name}'.")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{file_to_compare}'.")
except Exception as e:
    print(f"Lỗi: {e}")

Tổng số giá trị kiểm tra trong 'aid': 168
Số lượng giá trị trong 'AID': 173

Các giá trị trong 'aid' KHÔNG trùng khớp với giá trị nào trong 'AID':
1046.0
1050.0
1051.0
1060.0
1090.0
110.0
120.0
1400.0
1410.0
1425.0
1450.0
1451.0
1460.0
1465.0
1505.0
1515.0
1525.0
1545.0
1560.0
1565.0
160.0
1600.0
1650.0
1680.0
1699.0
1700.0
1751.0
1781.0
1790.0
195.0
2049.0
2075.0
2111.0
2112.0
2180.0
2181.0
2190.0
2199.0
2899.0
3005.0
3011.0
3012.0
3025.0
3070.0
3075.0
3077.0
3081.0
3082.0
3083.0
3084.0
3085.0
3086.0
3087.0
3088.0
3089.0
309.0
3090.0
3095.0
3099.0
3100.0
311.0
3185.0
3190.0
3325.0
3400.0
3407.0
349.0
352.0
3520.0
3521.0
356.0
3560.0
3561.0
3580.0
3590.0
399.0
4000.0
4001.0
4005.0
4020.0
4025.0
404.0
4040.0
4081.0
4085.0
4095.0
4097.0
4099.0
4100.0
4120.0
4150.0
4250.0
4400.0
4405.0
4520.0
4525.0
4590.0
4600.0
4605.0
4900.0
4910.0
4920.0
5025.0
5050.0
5052.0
5077.0
508.0
5080.0
5099.0
518.0
5181.0
5182.0
5270.0
5275.0
5290.0
5295.0
5300.0
5390.0
5500.0
6099.0
6300.0
6305.0
6310.0
6315.

: 

In [ ]:
import logging
import pyodbc
import pandas as pd
from datetime import datetime

mdb_file = r"C:\Users\gia.luongdo\Desktop\ERP-Importer\db_Artikel_Export2.mdb"
sku_basis_csv = r"C:\Users\gia.luongdo\Desktop\ERP-Importer\IMPORTER_SKU_Neuanlage_Basis.csv"
conn_str = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    f"DBQ={mdb_file};"
)
# Configure logging to write to a file
logging.basicConfig(
    filename='importer_log.txt',
    filemode='a',
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO)
# Connect to the Access database
conn = pyodbc.connect(conn_str)
logging.info("Connected to the database successfully.")
#IMPORTER_ARTICLE_Neuanlage_Basis:
#Reads data from the database
def read_and_write_aku_data():
    table_name = 't_Art_Mega_SKU'
    query = f"SELECT sku.ArtikelCode AS SKU, m.Ursprungsland, t.ArtBem AS name FROM (t_Art_Mega_SKU sku INNER JOIN t_Art_Text_DE t ON sku.ArtNr = t.ArtNr) INNER JOIN t_Art_MegaBase m ON sku.ArtNR = m.ArtNr WHERE m.Marke IN ('Corporate', 'EXCD', 'XO');"
    df = pd.read_sql(query, conn)

    # Add columns with specified values
    df['company'] = 0
    df['automatic_batch_numbering_pattern'] = '{No,000000000}'
    df['batch_management'] = 2
    df['batch_number_range'] = 'Chargen'
    df['batch_numbering_type'] = 3
    df['date_requirement'] = 1
    df['discountable'] = 'ja'
    df['factory'] = 'Düsseldorf'
    df['isPi'] = 'ja'
    df['isShopArticle'] = 'ja'
    df['isSl'] = 'ja'
    df['isSt'] = 'ja'
    df['isVerifiedArticle'] = 'ja'
    df['isCatalogArticle'] = 'ja'
    df['unitPi'] = 'Stk'
    df['unitSl'] = 'Stk'
    df['unitSt'] = 'Stk'
    #df['name'] = df['name']
    df['replacement_time'] = 1
    df['taxPi'] = 'Waren'
    df['taxSl'] = 'Waren'
    df['valid_from'] = datetime.now().strftime("%Y%m%d")

    # Take only first 2 characters from Ursprungsland and rename to 'country', reorder columns
    df['country_of_origin'] = df['Ursprungsland'].str[:2]
    df = df[['SKU', 'company', 'country_of_origin', 'automatic_batch_numbering_pattern', 'batch_management', 'batch_number_range', 'batch_numbering_type', 'date_requirement', 'discountable', 'factory', 'isPi', 'isShopArticle', 'isSl', 'isSt', 'isVerifiedArticle', 'isCatalogArticle', 'unitPi', 'unitSl', 'unitSt', 'name', 'replacement_time', 'taxPi', 'taxSl', 'valid_from']]

    # Write DataFrame to CSV with correct separator and columns
    df.to_csv(sku_basis_csv, index=False, encoding='utf-8', sep=',')
    print(f'Data exported to {sku_basis_csv}')
    logging.info(f'Data exported to {sku_basis_csv}')

# IMPORTER_SKU_CLASSIFICATION_Merkmale_Basis
def read_and_write_classification_data():
    """Reads classification data from the database and writes it to a CSV file."""
    # Define the table name and query
    classification_table_name = 't_Art_MegaBase'
    table_name = 't_Art_Flags'
    classification_query = f"""
        SELECT
            sku.ArtikelCode AS SKU,
            m.Ursprungsland AS Ursprungsland,
            sku.Größe AS Größe,
            sku.Größenspiegel AS Größenspiegel,
            sku.Hauptfarbe AS Farbgruppe,
            sku.FarbeNeu AS Farbe,
            sku.isColorCombination AS zweifarbig,
            m.Materialart AS Fabric_Herstellung,
            sku.ArtSort AS sku_ArtSort,
            m.Produktgruppe AS product_group,
            m.Marke AS Marke,
            m.Grammatur AS Grammatur,
            m.Artikel_Partner AS Artikel_Partner,
            m.ArtSort AS ArtSort,
            m.Zusammensetzung AS Zusammensetzung,
            m.Gender AS Gender,
            f.flag_workwear AS workwear,
            f.flag_veredelung AS veredelung,
            f.flag_discharge AS discharge,
            f.flag_dtg AS dtg,
            f.flag_dyoj AS dyoj,
            f.flag_dyop AS dyop,
            f.flag_flock AS flock,
            f.flag_siebdruck AS siebdruck,
            f.flag_stick AS stick,
            f.flag_sublimation AS sublimation,
            f.flag_transfer AS transfer,
            f.flag_premium AS premium,
            f.flag_extras AS extras,
            f.flag_outdoor AS outdoor,
            f.flag_plussize AS oversize,
            f.isNoLabel AS label,
            f.isErw AS erw,
            sku.Karton_Länge as Verpackungslänge,
            sku.Karton_Breite as Verpackungsbreite,
            sku.Karton_Höhe as Verpackungshoehe,
            sku.EAN as EAN,
            sku.Produktgewicht as Produktgewicht,
            sku.WarenNr as WarenNr
        FROM (t_Art_MegaBase m
        INNER JOIN t_Art_Flags f ON m.ArtNr = f.ArtNr )
        inner join t_Art_Mega_SKU sku on m.ArtNr = sku.ArtNr
        WHERE m.Marke IN ('Corporate', 'EXCD', 'XO')
    """
    classification_df = pd.read_sql(classification_query, conn)

    # Add columns with specified values
    classification_df['company'] = 0
    classification_df['classification_system'] = 'Warengruppensystem'
    classification_df['product_group_superior'] = classification_df['Marke'] + '||Produktlinie||ROOT'
    classification_df['ArtikelCode'] = classification_df['SKU']
    classification_df['feature[0]'] = 'Grammatur'
    classification_df['feature_value[0]'] = classification_df['Grammatur']
    classification_df['feature[1]'] = 'Oeko_MadeInGreen'
    classification_df['feature_value[1]'] = ''
    classification_df['feature[2]'] = 'Partnerlook'
    classification_df['feature_value[2]'] = classification_df['Artikel_Partner'].str[:4]
    classification_df['feature[3]'] = 'Sortierung'
    classification_df['feature_value[3]'] = classification_df['ArtSort']
    classification_df['feature[4]'] = 'Fabric_Herstellung'
    classification_df['feature_value[4]'] = classification_df['Fabric_Herstellung']
    classification_df['feature[5]'] = 'Material'
    classification_df['feature_value[5]'] = classification_df['Zusammensetzung']
    classification_df['feature[6]'] = 'Workwear'
    classification_df['feature_value[6]'] = abs(classification_df['workwear'])
    classification_df['feature[7]'] = 'Produktlinie_Veredelung'
    classification_df['feature_value[7]'] = abs(classification_df['veredelung'])
    classification_df['feature[8]'] = 'Produktlinie_Veredelungsart_Discharge'
    classification_df['feature_value[8]'] = abs(classification_df['discharge'])
    classification_df['feature[9]'] = 'Produktlinie_Veredelungsart_DTG'
    classification_df['feature_value[9]'] = abs(classification_df['dtg'])
    classification_df['feature[10]'] = 'Produktlinie_Veredelungsart_DYOJ'
    classification_df['feature_value[10]'] = abs(classification_df['dyoj'])
    classification_df['feature[11]'] = 'Produktlinie_Veredelungsart_DYOP'
    classification_df['feature_value[11]'] = abs(classification_df['dyop'])
    classification_df['feature[12]'] = 'Produktlinie_Veredelungsart_Flock'
    classification_df['feature_value[12]'] = abs(classification_df['flock'])
    classification_df['feature[13]'] = 'Produktlinie_Veredelungsart_Siebdruck'
    classification_df['feature_value[13]'] = abs(classification_df['siebdruck'])
    classification_df['feature[14]'] = 'Produktlinie_Veredelungsart_Stick'
    classification_df['feature_value[14]'] = abs(classification_df['stick'])
    classification_df['feature[15]'] = 'Produktlinie_Veredelungsart_Sublimationsdruck'
    classification_df['feature_value[15]'] = abs(classification_df['sublimation'])
    classification_df['feature[16]'] = 'Produktlinie_Veredelungsart_Transferdruck'
    classification_df['feature_value[16]'] = abs(classification_df['transfer'])
    classification_df['feature[17]'] = 'Brand_Premium_Item'
    classification_df['feature_value[17]'] = abs(classification_df['premium'])
    classification_df['feature[18]'] = 'Extras'
    classification_df['feature_value[18]'] = abs(classification_df['extras'])
    classification_df['feature[19]'] = 'Kids'
    classification_df['feature_value[19]'] = 1 - abs(classification_df['erw'])
    classification_df['feature[20]'] = 'Outdoor'
    classification_df['feature_value[20]'] = abs(classification_df['outdoor'])
    classification_df['feature[21]'] = 'Size_Oversize'
    classification_df['feature_value[21]'] = abs(classification_df['oversize'])
    classification_df['feature[22]'] = 'Geschlecht'
    classification_df['feature_value[22]'] = classification_df['Gender']
    classification_df['feature[23]'] = 'Brand_Label'
    classification_df['feature_value[23]'] = abs(classification_df['label'])
    classification_df['feature[24]'] = 'Farbe'
    classification_df['feature_value[24]'] = classification_df['Farbe']
    classification_df['feature[25]'] = 'Farbgruppe'
    classification_df['feature_value[25]'] = classification_df['Farbgruppe']
    classification_df['feature[26]'] = 'Größe'
    classification_df['feature_value[26]'] = classification_df['Größe']
    classification_df['feature[27]'] = 'Größenspiegel'
    classification_df['feature_value[27]'] = classification_df['Größenspiegel']
    classification_df['feature[28]'] = 'Zweifarbig'
    classification_df['feature_value[28]'] = classification_df['zweifarbig']
    classification_df['feature[29]'] = 'Ursprungsland'
    classification_df['feature_value[29]'] = classification_df['Ursprungsland'].str[:2]
    classification_df['feature[30]'] = 'Verpackungslänge'
    classification_df['feature_value[30]'] = classification_df['Verpackungslänge']
    classification_df['feature[31]'] = 'Verpackungsbreite'
    classification_df['feature_value[31]'] = classification_df['Verpackungsbreite']
    classification_df['feature[32]'] = 'Verpackungshoehe'
    classification_df['feature_value[32]'] = classification_df['Verpackungshoehe']
    classification_df['feature[33]'] = 'EAN'
    classification_df['feature_value[33]'] = classification_df['EAN']
    classification_df['feature[34]'] = 'Produktgewicht'
    classification_df['feature_value[34]'] = classification_df['Produktgewicht']
    classification_df['feature[35]'] = 'Statistische Warnennummer'
    classification_df['feature_value[35]'] = classification_df['WarenNr']
    # Reorder columns
    feature_cols = []
    for i in range(36):
        feature_cols.extend([f'feature[{i}]', f'feature_value[{i}]'])
    classification_df = classification_df[
        ['SKU', 'company', 'classification_system', 'product_group', 'product_group_superior'] + feature_cols
    ]

    # Write DataFrame to CSV (add your desired path)
    classification_csv = r"C:\Users\gia.luongdo\Desktop\ERP-Importer\IMPORTER_SKU_CLASSIFICATION_Classification_Basis.csv"
    classification_df.to_csv(
        classification_csv,
        index=False,
        encoding='windows-1252',
        sep=',',
    )
    print(f'Data exported to {classification_csv}')
    logging.info(f'Data exported to {classification_csv}')

#IMPORTER_SKU_Keyword
def read_and_write_SKU_Keyword():
    table_name = 't_Art_MegaBase'
    query = """
        SELECT 
            sku.ArtikelCode AS SKU,
            0 as company,
            t.SuchText as keyword_list,
            'de' as language,
            ',' as seperator
        FROM (t_Art_MegaBase m
        INNER JOIN t_Art_Mega_SKU sku ON m.ArtNr = sku.ArtNr)
        INNER JOIN t_Art_Text_DE t ON sku.ArtNr = t.ArtNr
        WHERE m.Marke IN ('Corporate', 'EXCD', 'XO')
    """
    df = pd.read_sql(query, conn)
    Zuordnung_csv = r"C:\Users\gia.luongdo\Desktop\ERP-Importer\IMPORTER_SKU_Keyword.csv"
    df.to_csv(Zuordnung_csv, index=False, encoding='windows-1252', sep=',')
    print(f'Data exported to {Zuordnung_csv}')
    logging.info(f'Data exported to {Zuordnung_csv}')

#Import_SKU_EAN
def read_and_write_SKU_EAN():
    table_name = 't_Art_MegaBase'
    query = """
        SELECT 
            sku.ArtikelCode AS SKU,
            0 as company,
            sku.EAN as ean,
            'de' as language,
            ',' as seperator
        FROM (t_Art_MegaBase m
        INNER JOIN t_Art_Mega_SKU sku ON m.ArtNr = sku.ArtNr)
        INNER JOIN t_Art_Text_DE t ON sku.ArtNr = t.ArtNr
        WHERE m.Marke IN ('Corporate', 'EXCD', 'XO')
    """
    df = pd.read_sql(query, conn)
    Zuordnung_csv = r"C:\Users\gia.luongdo\Desktop\ERP-Importer\IMPORTER_SKU_EAN.csv"
    df.to_csv(Zuordnung_csv, index=False, encoding='windows-1252', sep=',')
    print(f'Data exported to {Zuordnung_csv}')
    logging.info(f'Data exported to {Zuordnung_csv}')

#Link SKU and Article Basis
def link_sku_article_basis():
    query = f"SELECT sku.ArtikelCode AS SKU, m.ArtBasis as aid, m.ArtNr as ArtNr FROM t_Art_Mega_SKU sku INNER JOIN t_Art_MegaBase m ON sku.ArtNR = m.ArtNr WHERE m.Marke IN ('Corporate', 'EXCD', 'XO');"
    df = pd.read_sql(query, conn)

    df.to_csv(r"C:\Users\gia.luongdo\Desktop\ERP-Importer\IMPORTER_SKU_ArtBasis.csv", index=False, encoding='windows-1252', sep=',')

# Call the function to execute the logic





conn.close()

C:\Users\gia.luongdo\AppData\Local\Temp\ipykernel_25400\1300257503.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\gia.luongdo\AppData\Local\Temp\ipykernel_25400\1300257503.py:114: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  classification_df = pd.read_sql(classification_query, conn)


Data exported to C:\Users\gia.luongdo\Desktop\ERP-Importer\IMPORTER_SKU_Neuanlage_Basis.csv
Data exported to C:\Users\gia.luongdo\Desktop\ERP-Importer\IMPORTER_SKU_CLASSIFICATION_Classification_Basis.csv
Data exported to C:\Users\gia.luongdo\Desktop\ERP-Importer\IMPORTER_SKU_Keyword.csv
Data exported to C:\Users\gia.luongdo\Desktop\ERP-Importer\IMPORTER_SKU_EAN.csv


C:\Users\gia.luongdo\AppData\Local\Temp\ipykernel_25400\1300257503.py:227: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\gia.luongdo\AppData\Local\Temp\ipykernel_25400\1300257503.py:248: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\gia.luongdo\AppData\Local\Temp\ipykernel_25400\1300257503.py:257: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
